In [2]:
import pandas as pd
from configparser import ConfigParser
from sqlalchemy import create_engine
import numpy as np

In [3]:
def config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [4]:
param_dic=config()

connect = "postgresql+psycopg2://%s:%s@%s:%s/%s" % (
param_dic['user'],
param_dic['password'],
param_dic['host'],
param_dic['port'],
param_dic['database']
)

engine=create_engine(connect, echo=False)

connection = engine.connect()

In [5]:
my_query="select timestamp,close,high,low,open,trades,turnover,volume,vwap from bitmex as bt where  bt.timestamp >= date '2018-01-01' "

In [6]:
data= connection.execute(my_query).fetchall()

In [7]:
df=pd.DataFrame(data,columns=['timestamp','close','high','low','open','trades','turnover','volume','vwap'])


In [8]:
df.head()

,timestamp,close,high,low,open,trades,turnover,volume,vwap
0,2018-01-01 00:00:00+05:00,13925.5,13925.5,13892.5,13909.5,93,934907830,129990,13904.3
1,2018-01-01 00:01:00+05:00,13914.0,13938.5,13910.0,13925.5,143,3584829106,499411,13931.5
2,2018-01-01 00:02:00+05:00,13960.5,13979.5,13914.0,13914.0,196,2916432935,406988,13956.7
3,2018-01-01 00:03:00+05:00,13972.0,13972.0,13950.0,13960.5,76,862372954,120404,13962.6
4,2018-01-01 00:04:00+05:00,14010.5,14011.0,13968.5,13972.0,168,4861884435,680351,13993.8


In [9]:
df.isna().sum()/df.count()

timestamp    0.000000
close        0.000000
high         0.000000
low          0.000000
open         0.000000
trades       0.000000
turnover     0.000000
volume       0.000000
vwap         0.000219
dtype: float64

In [10]:
df['date']=df['timestamp'].dt.date

In [11]:
df=df.groupby('date').mean()

In [12]:
df['vwap'].replace(0, np.nan, inplace=True)
df['vwap'].fillna(method='ffill', inplace=True)

In [13]:
df.head()

,close,high,low,open,trades,turnover,volume,vwap
date,,,,,,,,
2018-01-01,13541.366319,13557.984028,13524.845139,13541.872917,151.901389,3.191650e+09,430376.524306,13542.399306
2018-01-02,13618.049306,13632.106597,13602.637153,13616.986458,173.356250,3.991163e+09,547527.584028,13618.215833
2018-01-03,14993.412500,15009.243750,14977.330556,14993.340278,188.893056,4.238429e+09,635719.903472,14994.467292
2018-01-04,14846.844444,14859.795139,14833.456944,14846.855556,158.861806,3.806899e+09,564605.880556,14847.653750
2018-01-05,15463.361806,15476.509722,15449.327083,15462.259028,164.459028,3.995719e+09,624329.748611,15464.094653


In [14]:
import numpy
from numpy import array
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,GRU
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.metrics import mean_squared_error
from pandas import Series

In [15]:
data=df['vwap']

In [16]:
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [169]:
np.random.seed(0)

# load the dataset
dataframe = data
dataset = dataframe.values
dataset = dataset.astype('float64').reshape(-1, 1)

# normalize the dataset
scaler = MinMaxScaler()
#scaler=RobustScaler()
#scaler=StandardScaler()
dataset = scaler.fit_transform(dataset)

# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

n_steps_in=4
n_steps_out=1

trainX, trainY =split_sequence(train.flatten(), n_steps_in, n_steps_out)
testX, testY = split_sequence(test.flatten(), n_steps_in, n_steps_out)


In [170]:
n_features = 1
trainX = trainX.reshape((trainX.shape[0], trainX.shape[1], n_features))
testX = testX.reshape((testX.shape[0], testX.shape[1], n_features))

In [171]:
trainX.shape,trainY.shape,testX.shape,testY.shape

((676, 4, 1), (676, 1), (332, 4, 1), (332, 1))

In [172]:
from keras.layers import Activation, Dense,Dropout
model = Sequential()

model.add(LSTM(15, return_sequences=True,input_shape=(n_steps_in, n_features)))
#model.add(Dropout(0.5))
model.add(LSTM(15))
#model.add(Dropout(0.5))

model.add(Dense(n_steps_out))

import keras
from keras import optimizers

#keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False, clipnorm=1)

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=300, verbose=1,shuffle=False,batch_size=50)


Epoch 1/300
14/14 [==============================] - 0s 3ms/step - loss: 0.1124
Epoch 2/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0720
Epoch 3/300
14/14 [==============================] - 0s 3ms/step - loss: 0.0398
Epoch 4/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0219
Epoch 5/300
14/14 [==============================] - 0s 5ms/step - loss: 0.0179
Epoch 6/300
14/14 [==============================] - 0s 6ms/step - loss: 0.0156
Epoch 7/300
14/14 [==============================] - 0s 5ms/step - loss: 0.0132
Epoch 8/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0113
Epoch 9/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0094
Epoch 10/300
14/14 [==============================] - 0s 5ms/step - loss: 0.0076
Epoch 11/300
14/14 [==============================] - 0s 5ms/step - loss: 0.0060
Epoch 12/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0047
Epoch 13/300
14/14 [=================

14/14 [==============================] - 0s 5ms/step - loss: 0.0015
Epoch 103/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0015
Epoch 104/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0015
Epoch 105/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0015
Epoch 106/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0015
Epoch 107/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0015
Epoch 108/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0015
Epoch 109/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0015
Epoch 110/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0014
Epoch 111/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0014
Epoch 112/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0014
Epoch 113/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0014
Epoch 114/300
14/14 [=========

14/14 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 203/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 204/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 205/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 206/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 207/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 208/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 209/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 210/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 211/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0012
Epoch 212/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0011
Epoch 213/300
14/14 [==============================] - 0s 4ms/step - loss: 0.0011
Epoch 214/300
14/14 [=========

14/14 [==============================] - 0s 4ms/step - loss: 6.4926e-04
Epoch 300/300
14/14 [==============================] - 0s 4ms/step - loss: 6.4662e-04


In [173]:
# make predictions
trainPredict = model.predict(trainX)



In [174]:
testPredict = model.predict(testX)

In [175]:
trainPredict.shape,testPredict.shape

((676, 1), (332, 1))

In [176]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(trainY)
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(testY)


In [177]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict))
print('Test Score: %.2f RMSE' % (testScore))
predictions = numpy.empty_like(dataset)
predictions[:, :] = numpy.nan
predictions[n_steps_in:len(trainPredict)+n_steps_in, :] = trainPredict
predictions[len(trainPredict)+(n_steps_in*2):len(dataset)+2, :] = testPredict

predictionsDF=pd.DataFrame(predictions,columns=["predicted"],index=dataframe.index)
ans=pd.concat([dataframe,predictionsDF],axis=1)
print( ans,[n_steps_in,trainScore,testScore])

Train Score: 330.27 RMSE
Test Score: 274.21 RMSE
                    vwap     predicted
date                                  
2018-01-01  13542.399306           NaN
2018-01-02  13618.215833           NaN
2018-01-03  14994.467292           NaN
2018-01-04  14847.653750           NaN
2018-01-05  15464.094653  14714.563477
...                  ...           ...
2020-10-08  10682.006324  10707.003906
2020-10-09  10945.438985  10754.613281
2020-10-10  11274.019792  10998.265625
2020-10-11  11361.710069  11336.256836
2020-10-12  11376.076368  11455.522461

[1016 rows x 2 columns] [4, 330.2661508327778, 274.2103458407201]
